In [43]:
import pandas as pd
import glob
import math

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_regimes(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for run in list(root.glob("run-*")):
        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [52]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = values.mean()
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                metric_values.append(math.ceil(mean_value*100)/100)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>Evaluation - Searching temperature hyperparameter</h1>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

<h2>PH2 Dataset</h2>

In [21]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

runs = {
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t_ft')),
}

In [22]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [23]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.00,0.347459,0.220506
1,H-UNet-SWTA,0,0.002,1.00,0.406330,0.272467
2,H-UNet-SWTA,0,0.005,1.00,0.376278,0.253494
3,H-UNet-SWTA,0,2.000,1.00,0.785376,0.659664
4,H-UNet-SWTA,0,5.000,1.00,0.794921,0.672899
5,H-UNet-SWTA,0,10.000,1.00,0.807764,0.689072
6,H-UNet-SWTA-FT,0,0.001,1.00,0.761935,0.632993
7,H-UNet-SWTA-FT,0,0.002,1.00,0.817542,0.703456
8,H-UNet-SWTA-FT,0,0.005,1.00,0.816057,0.702950
9,H-UNet-SWTA-FT,0,2.000,1.00,0.857253,0.758608


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.00    0.347459 NaN  0.220506 NaN
                 0.002    1.00    0.406330 NaN  0.272467 NaN
                 0.005    1.00    0.376278 NaN  0.253494 NaN
                 2.000    1.00    0.785376 NaN  0.659664 NaN
                 5.000    1.00    0.794921 NaN  0.672899 NaN
                 10.000   1.00    0.807764 NaN  0.689072 NaN
H-UNet-SWTA-FT   0.001    1.00    0.761935 NaN  0.632993 NaN
                 0.002    1.00    0.817542 NaN  0.703456 NaN
                 0.005    1.00    0.816057 NaN  0.702950 NaN
                 2.000    1.00    0.857253 NaN  0.758608 NaN
                 5.000    1.00    0.860508 NaN  0.766551 NaN
                 10.000   1.00    0.857023 NaN  0.763188 NaN
H-UNet-SWTA-T    0.001    1.00    0.364194 NaN  0.236520 NaN
                 0.002    1.00    0.500131 NaN  0.366627 NaN
                 0.005    1.00    0.344096 NaN  0.216778 NaN
                 2.000    1.00    0.798013 NaN  0.674931 NaN
                 5.000    1.00    0.811419 NaN  0.695058 NaN
                 10.000   1.00    0.798698 NaN  0.676844 NaN
H-UNet-SWTA-T-FT 0.001    1.00    0.781603 NaN  0.658246 NaN
                 0.002    1.00    0.793119 NaN  0.672910 NaN
                 0.005    1.00    0.779779 NaN  0.654959 NaN
                 2.000    1.00    0.851924 NaN  0.754643 NaN
                 5.000    1.00    0.866120 NaN  0.774140 NaN
                 10.000   1.00    0.853715 NaN  0.755382 NaN
UNet             1.000    0.05    0.792022 NaN  0.678416 NaN
                          0.10    0.753338 NaN  0.628216 NaN
                          0.25    0.791293 NaN  0.674713 NaN
                          0.50    0.855957 NaN  0.762099 NaN
                          0.75    0.830643 NaN  0.728687 NaN
                          1.00    0.851703 NaN  0.756285 NaN

<h2>ISIC 2016 Dataset</h2>

In [4]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

runs = {
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=isic2016/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=isic2016/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/').glob('hunet-swta_t_ft')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.279620,0.171027
1,H-UNet-SWTA,0,0.002,1.0,0.367899,0.237425
2,H-UNet-SWTA,0,0.005,1.0,0.473021,0.334356
3,H-UNet-SWTA,0,2.000,1.0,0.737769,0.609246
4,H-UNet-SWTA,0,5.000,1.0,0.730096,0.597795
5,H-UNet-SWTA,0,10.000,1.0,0.743858,0.623220
6,H-UNet-SWTA-FT,0,0.001,1.0,0.820996,0.713543
7,H-UNet-SWTA-FT,0,0.002,1.0,0.778024,0.657678
8,H-UNet-SWTA-FT,0,0.005,1.0,0.734156,0.600639
9,H-UNet-SWTA-FT,0,2.000,1.0,0.848543,0.749035


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.279620 NaN  0.171027 NaN
                 0.002    1.0     0.367899 NaN  0.237425 NaN
                 0.005    1.0     0.473021 NaN  0.334356 NaN
                 2.000    1.0     0.737769 NaN  0.609246 NaN
                 5.000    1.0     0.730096 NaN  0.597795 NaN
                 10.000   1.0     0.743858 NaN  0.623220 NaN
H-UNet-SWTA-FT   0.001    1.0     0.820996 NaN  0.713543 NaN
                 0.002    1.0     0.778024 NaN  0.657678 NaN
                 0.005    1.0     0.734156 NaN  0.600639 NaN
                 2.000    1.0     0.848543 NaN  0.749035 NaN
                 5.000    1.0     0.846684 NaN  0.749667 NaN
                 10.000   1.0     0.837474 NaN  0.735330 NaN
H-UNet-SWTA-T    0.001    1.0     0.498905 NaN  0.355979 NaN
                 0.002    1.0     0.318617 NaN  0.199275 NaN
                 0.005    1.0     0.267750 NaN  0.163044 NaN
                 2.000    1.0     0.737928 NaN  0.607723 NaN
                 5.000    1.0     0.740001 NaN  0.611534 NaN
                 10.000   1.0     0.740710 NaN  0.616802 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.751068 NaN  0.625782 NaN
                 0.002    1.0     0.807158 NaN  0.692174 NaN
                 0.005    1.0     0.829815 NaN  0.723643 NaN
                 2.000    1.0     0.838178 NaN  0.735655 NaN
                 5.000    1.0     0.810767 NaN  0.710726 NaN
                 10.000   1.0     0.848248 NaN  0.747647 NaN
UNet             1.000    1.0     0.836816 NaN  0.735125 NaN

<h2>KvasirSEG Dataset</h2>

In [25]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

runs = {
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_t_ft')),
}

In [26]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [27]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.261240,0.156116
1,H-UNet-SWTA,0,0.002,1.0,0.277637,0.170643
2,H-UNet-SWTA,0,0.005,1.0,0.264526,0.158610
3,H-UNet-SWTA,0,2.000,1.0,0.406727,0.272606
4,H-UNet-SWTA,0,5.000,1.0,0.409068,0.273103
5,H-UNet-SWTA,0,10.000,1.0,0.411893,0.276746
6,H-UNet-SWTA-FT,0,0.001,1.0,0.408150,0.275266
7,H-UNet-SWTA-FT,0,0.002,1.0,0.380882,0.250689
8,H-UNet-SWTA-FT,0,0.005,1.0,0.390853,0.258718
9,H-UNet-SWTA-FT,0,2.000,1.0,0.735841,0.616942


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.261240 NaN  0.156116 NaN
                 0.002    1.0     0.277637 NaN  0.170643 NaN
                 0.005    1.0     0.264526 NaN  0.158610 NaN
                 2.000    1.0     0.406727 NaN  0.272606 NaN
                 5.000    1.0     0.409068 NaN  0.273103 NaN
                 10.000   1.0     0.411893 NaN  0.276746 NaN
H-UNet-SWTA-FT   0.001    1.0     0.408150 NaN  0.275266 NaN
                 0.002    1.0     0.380882 NaN  0.250689 NaN
                 0.005    1.0     0.390853 NaN  0.258718 NaN
                 2.000    1.0     0.735841 NaN  0.616942 NaN
                 5.000    1.0     0.741005 NaN  0.620023 NaN
                 10.000   1.0     0.725275 NaN  0.604296 NaN
H-UNet-SWTA-T    0.001    1.0     0.273035 NaN  0.164780 NaN
                 0.002    1.0     0.296798 NaN  0.184530 NaN
                 0.005    1.0     0.255037 NaN  0.151997 NaN
                 2.000    1.0     0.406665 NaN  0.270880 NaN
                 5.000    1.0     0.408579 NaN  0.271835 NaN
                 10.000   1.0     0.434897 NaN  0.295786 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.384478 NaN  0.252906 NaN
                 0.002    1.0     0.400953 NaN  0.266831 NaN
                 0.005    1.0     0.393808 NaN  0.259085 NaN
                 2.000    1.0     0.701072 NaN  0.577696 NaN
                 5.000    1.0     0.741180 NaN  0.624408 NaN
                 10.000   1.0     0.745406 NaN  0.627872 NaN
UNet             1.000    1.0     0.685022 NaN  0.560719 NaN

<h2>Data Science Bowl 2018 Dataset</h2>

In [28]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

runs = {
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/').glob('hunet-swta_t_ft')),
}

In [29]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [30]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.433182,0.293950
1,H-UNet-SWTA,0,0.002,1.0,0.633607,0.479857
2,H-UNet-SWTA,0,0.005,1.0,0.169068,0.097866
3,H-UNet-SWTA,0,2.000,1.0,0.759079,0.622942
4,H-UNet-SWTA,0,5.000,1.0,0.752148,0.617175
5,H-UNet-SWTA,0,10.000,1.0,0.780533,0.653930
6,H-UNet-SWTA-FT,0,0.001,1.0,0.809131,0.692046
7,H-UNet-SWTA-FT,0,0.002,1.0,0.779566,0.657712
8,H-UNet-SWTA-FT,0,0.005,1.0,0.731529,0.598980
9,H-UNet-SWTA-FT,0,2.000,1.0,0.824140,0.713056


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.433182 NaN  0.293950 NaN
                 0.002    1.0     0.633607 NaN  0.479857 NaN
                 0.005    1.0     0.169068 NaN  0.097866 NaN
                 2.000    1.0     0.759079 NaN  0.622942 NaN
                 5.000    1.0     0.752148 NaN  0.617175 NaN
                 10.000   1.0     0.780533 NaN  0.653930 NaN
H-UNet-SWTA-FT   0.001    1.0     0.809131 NaN  0.692046 NaN
                 0.002    1.0     0.779566 NaN  0.657712 NaN
                 0.005    1.0     0.731529 NaN  0.598980 NaN
                 2.000    1.0     0.824140 NaN  0.713056 NaN
                 5.000    1.0     0.833603 NaN  0.728412 NaN
                 10.000   1.0     0.832328 NaN  0.729832 NaN
H-UNet-SWTA-T    0.001    1.0     0.606491 NaN  0.466019 NaN
                 0.002    1.0     0.656834 NaN  0.515459 NaN
                 0.005    1.0     0.636921 NaN  0.492823 NaN
                 2.000    1.0     0.748851 NaN  0.611852 NaN
                 5.000    1.0     0.752805 NaN  0.616452 NaN
                 10.000   1.0     0.731121 NaN  0.592810 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.805338 NaN  0.684946 NaN
                 0.002    1.0     0.787010 NaN  0.661773 NaN
                 0.005    1.0     0.823677 NaN  0.710350 NaN
                 2.000    1.0     0.832033 NaN  0.723323 NaN
                 5.000    1.0     0.801898 NaN  0.686990 NaN
                 10.000   1.0     0.828268 NaN  0.717549 NaN
UNet             1.000    1.0     0.837379 NaN  0.729981 NaN

<h2>GlaS Dataset</h2>

In [56]:
EXP_ROOT = "./runs"
#EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

runs = {
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft')),
}

In [57]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [58]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,5.0,1.0,68.71,53.41,48.13,6.91
1,H-UNet-BASE-SWTA,0,10.0,1.0,70.88,55.86,39.84,5.32
2,H-UNet-BASE-SWTA,0,12.0,1.0,70.58,55.18,22.02,3.09
3,H-UNet-BASE-SWTA-FT,0,5.0,1.0,86.42,76.80,17.13,2.23
4,H-UNet-BASE-SWTA-FT,0,10.0,1.0,87.94,79.15,14.16,1.97
5,H-UNet-BASE-SWTA-FT,0,12.0,1.0,87.07,77.80,13.28,1.93
6,H-UNet-BASE-SWTA-T,0,5.0,1.0,68.90,53.46,34.26,4.79
7,H-UNet-BASE-SWTA-T,0,10.0,1.0,71.49,56.58,31.18,4.10
8,H-UNet-BASE-SWTA-T,0,12.0,1.0,71.12,55.84,20.93,2.75
9,H-UNet-BASE-SWTA-T-FT,0,5.0,1.0,87.47,78.44,16.97,2.20


Dice     Jaccard      \
                                        mean std    mean std   
Model                 Inv Temp Regime                          
H-UNet-BASE-SWTA      5.0      1.0     68.71 NaN   53.41 NaN   
                      10.0     1.0     70.88 NaN   55.86 NaN   
                      12.0     1.0     70.58 NaN   55.18 NaN   
H-UNet-BASE-SWTA-FT   5.0      1.0     86.42 NaN   76.80 NaN   
                      10.0     1.0     87.94 NaN   79.15 NaN   
                      12.0     1.0     87.07 NaN   77.80 NaN   
H-UNet-BASE-SWTA-T    5.0      1.0     68.90 NaN   53.46 NaN   
                      10.0     1.0     71.49 NaN   56.58 NaN   
                      12.0     1.0     71.12 NaN   55.84 NaN   
H-UNet-BASE-SWTA-T-FT 5.0      1.0     87.47 NaN   78.44 NaN   
                      10.0     1.0     87.60 NaN   78.67 NaN   
                      12.0     1.0     86.55 NaN   77.14 NaN   

                                      Hausdorff Distance      \
                                                    mean std   
Model                 Inv Temp Regime                          
H-UNet-BASE-SWTA      5.0      1.0                 48.13 NaN   
                      10.0     1.0                 39.84 NaN   
                      12.0     1.0                 22.02 NaN   
H-UNet-BASE-SWTA-FT   5.0      1.0                 17.13 NaN   
                      10.0     1.0                 14.16 NaN   
                      12.0     1.0                 13.28 NaN   
H-UNet-BASE-SWTA-T    5.0      1.0                 34.26 NaN   
                      10.0     1.0                 31.18 NaN   
                      12.0     1.0                 20.93 NaN   
H-UNet-BASE-SWTA-T-FT 5.0      1.0                 16.97 NaN   
                      10.0     1.0                 13.87 NaN   
                      12.0     1.0                 14.45 NaN   

                                      Average Surface Distance      
                                                          mean std  
Model                 Inv Temp Regime                               
H-UNet-BASE-SWTA      5.0      1.0                        6.91 NaN  
                      10.0     1.0                        5.32 NaN  
                      12.0     1.0                        3.09 NaN  
H-UNet-BASE-SWTA-FT   5.0      1.0                        2.23 NaN  
                      10.0     1.0                        1.97 NaN  
                      12.0     1.0                        1.93 NaN  
H-UNet-BASE-SWTA-T    5.0      1.0                        4.79 NaN  
                      10.0     1.0                        4.10 NaN  
                      12.0     1.0                        2.75 NaN  
H-UNet-BASE-SWTA-T-FT 5.0      1.0                        2.20 NaN  
                      10.0     1.0                        1.93 NaN  
                      12.0     1.0                        2.23 NaN

<h1>Evaluation - Hebbian Models</h1>

<p>Evaluate Hebbian models over the datasets; only regime=1.0 and best temperature values for SWTA are considered.</p>

<h2>PH2 Dataset</h2>

In [4]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_PH2=5          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=ph2/unet/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=ph2/fcn32s/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-FCN32s-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run*')),
    'H-FCN32s-HPCA': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
}

In [25]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [26]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,1.0,0.857434,0.763633
1,H-FCN32s-HPCA,0,1.0,1.0,0.854888,0.757898
2,H-FCN32s-HPCA-FT,0,1.0,1.0,0.873817,0.789046
3,H-FCN32s-HPCA-T,0,1.0,1.0,0.848687,0.752533
4,H-FCN32s-HPCA-T-FT,0,1.0,1.0,0.866091,0.779639
5,H-FCN32s-SWTA,0,5.0,1.0,0.845275,0.744243
6,H-FCN32s-SWTA-FT,0,5.0,1.0,0.869143,0.782203
7,H-FCN32s-SWTA-T,0,5.0,1.0,0.852601,0.753760
8,H-FCN32s-SWTA-T-FT,0,5.0,1.0,0.874096,0.785881
9,H-UNet-HPCA,0,1.0,1.0,0.606867,0.447543


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      1.0     0.857434 NaN  0.763633 NaN
H-FCN32s-HPCA      1.0      1.0     0.854888 NaN  0.757898 NaN
H-FCN32s-HPCA-FT   1.0      1.0     0.873817 NaN  0.789046 NaN
H-FCN32s-HPCA-T    1.0      1.0     0.848687 NaN  0.752533 NaN
H-FCN32s-HPCA-T-FT 1.0      1.0     0.866091 NaN  0.779639 NaN
H-FCN32s-SWTA      5.0      1.0     0.845275 NaN  0.744243 NaN
H-FCN32s-SWTA-FT   5.0      1.0     0.869143 NaN  0.782203 NaN
H-FCN32s-SWTA-T    5.0      1.0     0.852601 NaN  0.753760 NaN
H-FCN32s-SWTA-T-FT 5.0      1.0     0.874096 NaN  0.785881 NaN
H-UNet-HPCA        1.0      1.0     0.606867 NaN  0.447543 NaN
H-UNet-HPCA-FT     1.0      1.0     0.861720 NaN  0.771171 NaN
H-UNet-HPCA-T      1.0      1.0     0.584115 NaN  0.432125 NaN
H-UNet-HPCA-T-FT   1.0      1.0     0.864442 NaN  0.772999 NaN
H-UNet-SWTA        5.0      1.0     0.794921 NaN  0.672899 NaN
H-UNet-SWTA-FT     5.0      1.0     0.860508 NaN  0.766551 NaN
H-UNet-SWTA-T      5.0      1.0     0.811419 NaN  0.695058 NaN
H-UNet-SWTA-T-FT   5.0      1.0     0.866120 NaN  0.774140 NaN
UNet               1.0      1.0     0.851703 NaN  0.756285 NaN

<h2>ISIC 2016 Dataset</h2>

In [22]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_ISIC2016=2          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=isic2016/unet/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=isic2016/fcn32s/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-SWTA': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-FCN32s-SWTA-T': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_ISIC2016)).glob('run*')),
    'H-FCN32s-HPCA': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
}

In [23]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [24]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,1.0,0.872915,0.781819
1,H-FCN32s-HPCA,0,1.0,1.0,0.866247,0.772063
2,H-FCN32s-HPCA-FT,0,1.0,1.0,0.879861,0.792006
3,H-FCN32s-HPCA-T,0,1.0,1.0,0.867128,0.774116
4,H-FCN32s-HPCA-T-FT,0,1.0,1.0,0.881259,0.795281
5,H-FCN32s-SWTA,0,2.0,1.0,0.869636,0.777014
6,H-FCN32s-SWTA-FT,0,2.0,1.0,0.883407,0.797501
7,H-FCN32s-SWTA-T,0,2.0,1.0,0.860046,0.762902
8,H-FCN32s-SWTA-T-FT,0,2.0,1.0,0.876768,0.787245
9,H-UNet-HPCA,0,1.0,1.0,0.366986,0.240006


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      1.0     0.872915 NaN  0.781819 NaN
H-FCN32s-HPCA      1.0      1.0     0.866247 NaN  0.772063 NaN
H-FCN32s-HPCA-FT   1.0      1.0     0.879861 NaN  0.792006 NaN
H-FCN32s-HPCA-T    1.0      1.0     0.867128 NaN  0.774116 NaN
H-FCN32s-HPCA-T-FT 1.0      1.0     0.881259 NaN  0.795281 NaN
H-FCN32s-SWTA      2.0      1.0     0.869636 NaN  0.777014 NaN
H-FCN32s-SWTA-FT   2.0      1.0     0.883407 NaN  0.797501 NaN
H-FCN32s-SWTA-T    2.0      1.0     0.860046 NaN  0.762902 NaN
H-FCN32s-SWTA-T-FT 2.0      1.0     0.876768 NaN  0.787245 NaN
H-UNet-HPCA        1.0      1.0     0.366986 NaN  0.240006 NaN
H-UNet-HPCA-FT     1.0      1.0     0.253949 NaN  0.169041 NaN
H-UNet-HPCA-T      1.0      1.0     0.582345 NaN  0.431686 NaN
H-UNet-HPCA-T-FT   1.0      1.0     0.824910 NaN  0.721946 NaN
H-UNet-SWTA        2.0      1.0     0.737769 NaN  0.609246 NaN
H-UNet-SWTA-FT     2.0      1.0     0.848543 NaN  0.749035 NaN
H-UNet-SWTA-T      2.0      1.0     0.737928 NaN  0.607723 NaN
H-UNet-SWTA-T-FT   2.0      1.0     0.838178 NaN  0.735655 NaN
UNet               1.0      1.0     0.836816 NaN  0.735125 NaN

<h2>KvasirSEG Dataset</h2>

In [30]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_KvasirSEG=5          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=kvasirSEG/unet/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=kvasirSEG/fcn32s/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-FCN32s-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_KvasirSEG)).glob('run*')),
    'H-FCN32s-HPCA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
}

In [31]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [32]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,1.0,0.732239,0.615262
1,H-FCN32s-HPCA,0,1.0,1.0,0.727332,0.611378
2,H-FCN32s-HPCA-FT,0,1.0,1.0,0.764679,0.658036
3,H-FCN32s-HPCA-T,0,1.0,1.0,0.719719,0.604748
4,H-FCN32s-HPCA-T-FT,0,1.0,1.0,0.766556,0.660787
5,H-FCN32s-SWTA,0,5.0,1.0,0.756055,0.640301
6,H-FCN32s-SWTA-FT,0,5.0,1.0,0.794742,0.690319
7,H-FCN32s-SWTA-T,0,5.0,1.0,0.735046,0.617083
8,H-FCN32s-SWTA-T-FT,0,5.0,1.0,0.782000,0.676947
9,H-UNet-HPCA,0,1.0,1.0,0.236932,0.141661


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      1.0     0.732239 NaN  0.615262 NaN
H-FCN32s-HPCA      1.0      1.0     0.727332 NaN  0.611378 NaN
H-FCN32s-HPCA-FT   1.0      1.0     0.764679 NaN  0.658036 NaN
H-FCN32s-HPCA-T    1.0      1.0     0.719719 NaN  0.604748 NaN
H-FCN32s-HPCA-T-FT 1.0      1.0     0.766556 NaN  0.660787 NaN
H-FCN32s-SWTA      5.0      1.0     0.756055 NaN  0.640301 NaN
H-FCN32s-SWTA-FT   5.0      1.0     0.794742 NaN  0.690319 NaN
H-FCN32s-SWTA-T    5.0      1.0     0.735046 NaN  0.617083 NaN
H-FCN32s-SWTA-T-FT 5.0      1.0     0.782000 NaN  0.676947 NaN
H-UNet-HPCA        1.0      1.0     0.236932 NaN  0.141661 NaN
H-UNet-HPCA-FT     1.0      1.0     0.274454 NaN  0.173328 NaN
H-UNet-HPCA-T      1.0      1.0     0.263937 NaN  0.157540 NaN
H-UNet-HPCA-T-FT   1.0      1.0     0.699386 NaN  0.574928 NaN
H-UNet-SWTA        5.0      1.0     0.409068 NaN  0.273103 NaN
H-UNet-SWTA-FT     5.0      1.0     0.741005 NaN  0.620023 NaN
H-UNet-SWTA-T      5.0      1.0     0.408579 NaN  0.271835 NaN
H-UNet-SWTA-T-FT   5.0      1.0     0.741180 NaN  0.624408 NaN
UNet               1.0      1.0     0.685022 NaN  0.560719 NaN

<h2>Data Science Bowl 2018 Dataset</h2>

In [39]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_DataScienceBowl2018=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/unet/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-SWTA': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/fcn32s/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-SWTA': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-swta_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-FCN32s-SWTA-T': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-swta_t_ft/inv_temp-{}/regime-1.0'.format(INV_TEMP_DataScienceBowl2018)).glob('run*')),
    'H-FCN32s-HPCA': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-hpca/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-hpca_ft/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-hpca_t/inv_temp-1/regime-1.0').glob('run*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-hpca_t_ft/inv_temp-1/regime-1.0').glob('run*')),
}

In [40]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [41]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,1.0,0.746234,0.611988
1,H-FCN32s-HPCA,0,1.0,1.0,0.716620,0.577560
2,H-FCN32s-HPCA-FT,0,1.0,1.0,0.784804,0.660039
3,H-FCN32s-HPCA-T,0,1.0,1.0,0.722426,0.583222
4,H-FCN32s-HPCA-T-FT,0,1.0,1.0,0.780666,0.654993
5,H-FCN32s-SWTA,0,10.0,1.0,0.719685,0.579699
6,H-FCN32s-SWTA-FT,0,10.0,1.0,0.778396,0.651899
7,H-FCN32s-SWTA-T,0,10.0,1.0,0.713937,0.574011
8,H-FCN32s-SWTA-T-FT,0,10.0,1.0,0.779170,0.652698
9,H-UNet-HPCA,0,1.0,1.0,0.189120,0.107064


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      1.0     0.746234 NaN  0.611988 NaN
H-FCN32s-HPCA      1.0      1.0     0.716620 NaN  0.577560 NaN
H-FCN32s-HPCA-FT   1.0      1.0     0.784804 NaN  0.660039 NaN
H-FCN32s-HPCA-T    1.0      1.0     0.722426 NaN  0.583222 NaN
H-FCN32s-HPCA-T-FT 1.0      1.0     0.780666 NaN  0.654993 NaN
H-FCN32s-SWTA      10.0     1.0     0.719685 NaN  0.579699 NaN
H-FCN32s-SWTA-FT   10.0     1.0     0.778396 NaN  0.651899 NaN
H-FCN32s-SWTA-T    10.0     1.0     0.713937 NaN  0.574011 NaN
H-FCN32s-SWTA-T-FT 10.0     1.0     0.779170 NaN  0.652698 NaN
H-UNet-HPCA        1.0      1.0     0.189120 NaN  0.107064 NaN
H-UNet-HPCA-FT     1.0      1.0     0.800751 NaN  0.679724 NaN
H-UNet-HPCA-T      1.0      1.0     0.504809 NaN  0.352018 NaN
H-UNet-HPCA-T-FT   1.0      1.0     0.828157 NaN  0.716986 NaN
H-UNet-SWTA        10.0     1.0     0.780533 NaN  0.653930 NaN
H-UNet-SWTA-FT     10.0     1.0     0.832328 NaN  0.729832 NaN
H-UNet-SWTA-T      10.0     1.0     0.731121 NaN  0.592810 NaN
H-UNet-SWTA-T-FT   10.0     1.0     0.828268 NaN  0.717549 NaN
UNet               1.0      1.0     0.837379 NaN  0.729981 NaN

<h1>Evaluation - Data regime variations</h1>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

<h2>PH2 Dataset</h2>

In [15]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_PH2=5          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=ph2/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=ph2/fcn32s/inv_temp-1').glob('regime-*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hfcn32s-hpca_t_ft/inv_temp-1').glob('regime-*')),
}

In [16]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [17]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,0.05,0.642098,0.488886
1,FCN32s,0,1.0,0.10,0.785834,0.660414
2,FCN32s,0,1.0,0.25,0.791358,0.669605
3,FCN32s,0,1.0,0.50,0.845848,0.744026
4,FCN32s,0,1.0,0.75,0.804273,0.687754
5,FCN32s,0,1.0,1.00,0.857434,0.763633
6,H-FCN32s-HPCA-FT,0,1.0,0.05,0.757700,0.642678
7,H-FCN32s-HPCA-FT,0,1.0,0.10,0.840199,0.739049
8,H-FCN32s-HPCA-FT,0,1.0,0.25,0.826667,0.723607
9,H-FCN32s-HPCA-FT,0,1.0,0.50,0.855222,0.760780


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      0.05    0.642098 NaN  0.488886 NaN
                            0.10    0.785834 NaN  0.660414 NaN
                            0.25    0.791358 NaN  0.669605 NaN
                            0.50    0.845848 NaN  0.744026 NaN
                            0.75    0.804273 NaN  0.687754 NaN
                            1.00    0.857434 NaN  0.763633 NaN
H-FCN32s-HPCA-FT   1.0      0.05    0.757700 NaN  0.642678 NaN
                            0.10    0.840199 NaN  0.739049 NaN
                            0.25    0.826667 NaN  0.723607 NaN
                            0.50    0.855222 NaN  0.760780 NaN
                            0.75    0.876711 NaN  0.790430 NaN
                            1.00    0.873817 NaN  0.789046 NaN
H-FCN32s-HPCA-T-FT 1.0      0.05    0.764061 NaN  0.644822 NaN
                            0.10    0.803820 NaN  0.693853 NaN
                            0.25    0.843387 NaN  0.739776 NaN
                            0.50    0.855855 NaN  0.759370 NaN
                            0.75    0.860954 NaN  0.772149 NaN
                            1.00    0.866091 NaN  0.779639 NaN
H-FCN32s-SWTA-FT   5.0      0.05    0.774119 NaN  0.656599 NaN
                            0.10    0.792900 NaN  0.677495 NaN
                            0.25    0.842910 NaN  0.744914 NaN
                            0.50    0.857701 NaN  0.761497 NaN
                            0.75    0.865245 NaN  0.771531 NaN
                            1.00    0.869143 NaN  0.782203 NaN
H-FCN32s-SWTA-T-FT 5.0      0.05    0.733974 NaN  0.595799 NaN
                            0.10    0.816103 NaN  0.703936 NaN
                            0.25    0.837419 NaN  0.735936 NaN
                            0.50    0.858403 NaN  0.765791 NaN
                            0.75    0.876022 NaN  0.788038 NaN
                            1.00    0.874096 NaN  0.785881 NaN
H-UNet-HPCA-FT     1.0      0.05    0.630311 NaN  0.511293 NaN
                            0.10    0.673933 NaN  0.555515 NaN
                            0.25    0.778379 NaN  0.654750 NaN
                            0.50    0.849762 NaN  0.754222 NaN
                            0.75    0.847608 NaN  0.748687 NaN
                            1.00    0.861720 NaN  0.771171 NaN
H-UNet-HPCA-T-FT   1.0      0.05    0.683679 NaN  0.554894 NaN
                            0.10    0.690923 NaN  0.571784 NaN
                            0.25    0.843274 NaN  0.744811 NaN
                            0.50    0.825000 NaN  0.714200 NaN
                            0.75    0.864205 NaN  0.773473 NaN
                            1.00    0.864442 NaN  0.772999 NaN
H-UNet-SWTA-FT     5.0      0.05    0.655645 NaN  0.527902 NaN
                            0.10    0.804325 NaN  0.685762 NaN
                            0.25    0.820000 NaN  0.712299 NaN
                            0.50    0.839729 NaN  0.739437 NaN
                            0.75    0.858459 NaN  0.764132 NaN
                            1.00    0.860508 NaN  0.766551 NaN
H-UNet-SWTA-T-FT   5.0      0.05    0.738578 NaN  0.624489 NaN
                            0.10    0.781450 NaN  0.657550 NaN
                            0.25    0.848454 NaN  0.745193 NaN
                            0.50    0.850957 NaN  0.754186 NaN
                            0.75    0.858806 NaN  0.764058 NaN
                            1.00    0.866120 NaN  0.774140 NaN
UNet               1.0      0.05    0.792022 NaN  0.678416 NaN
                            0.10    0.753338 NaN  0.628216 NaN
                            0.25    0.791293 NaN  0.674713 NaN
                            0.50    0.855957 NaN  0.762099 NaN
                            0.75    0.830643 NaN  0.728687 NaN
                            1.00    0.851703 NaN  0.756285 NaN

<h2>ISIC 2016 Dataset</h2>

In [6]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_ISIC2016=2          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=isic2016/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_ISIC2016)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_ISIC2016)).glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=isic2016/fcn32s/inv_temp-1').glob('regime-*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-swta_ft/inv_temp-{}'.format(INV_TEMP_ISIC2016)).glob('regime-*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-swta_t_ft/inv_temp-{}'.format(INV_TEMP_ISIC2016)).glob('regime-*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=isic2016/hfcn32s-hpca_t_ft/inv_temp-1').glob('regime-*')),
}

In [7]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [8]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,0.05,0.707442,0.576727
1,FCN32s,0,1.0,0.10,0.780179,0.662294
2,FCN32s,0,1.0,0.25,0.824996,0.717826
3,FCN32s,0,1.0,0.50,0.856487,0.758222
4,FCN32s,0,1.0,0.75,0.868695,0.776586
5,FCN32s,0,1.0,1.00,0.872915,0.781819
6,H-FCN32s-HPCA-FT,0,1.0,0.05,0.850167,0.754304
7,H-FCN32s-HPCA-FT,0,1.0,0.10,0.851584,0.753386
8,H-FCN32s-HPCA-FT,0,1.0,0.25,0.871295,0.780524
9,H-FCN32s-HPCA-FT,0,1.0,0.50,0.874527,0.784787


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      0.05    0.707442 NaN  0.576727 NaN
                            0.10    0.780179 NaN  0.662294 NaN
                            0.25    0.824996 NaN  0.717826 NaN
                            0.50    0.856487 NaN  0.758222 NaN
                            0.75    0.868695 NaN  0.776586 NaN
                            1.00    0.872915 NaN  0.781819 NaN
H-FCN32s-HPCA-FT   1.0      0.05    0.850167 NaN  0.754304 NaN
                            0.10    0.851584 NaN  0.753386 NaN
                            0.25    0.871295 NaN  0.780524 NaN
                            0.50    0.874527 NaN  0.784787 NaN
                            0.75    0.877072 NaN  0.788622 NaN
                            1.00    0.879861 NaN  0.792006 NaN
H-FCN32s-HPCA-T-FT 1.0      0.05    0.867569 NaN  0.776068 NaN
                            0.10    0.867415 NaN  0.775801 NaN
                            0.25    0.875360 NaN  0.789370 NaN
                            0.50    0.881367 NaN  0.794667 NaN
                            0.75    0.879660 NaN  0.792640 NaN
                            1.00    0.881259 NaN  0.795281 NaN
H-FCN32s-SWTA-FT   2.0      0.05    0.874085 NaN  0.786474 NaN
                            0.10    0.873055 NaN  0.783751 NaN
                            0.25    0.882693 NaN  0.797139 NaN
                            0.50    0.876615 NaN  0.787834 NaN
                            0.75    0.883104 NaN  0.797109 NaN
                            1.00    0.883407 NaN  0.797501 NaN
H-FCN32s-SWTA-T-FT 2.0      0.05    0.859832 NaN  0.763857 NaN
                            0.10    0.853163 NaN  0.756936 NaN
                            0.25    0.868440 NaN  0.776861 NaN
                            0.50    0.870375 NaN  0.778650 NaN
                            0.75    0.875196 NaN  0.785353 NaN
                            1.00    0.876768 NaN  0.787245 NaN
H-UNet-HPCA-FT     1.0      0.05    0.265133 NaN  0.163299 NaN
                            0.10         NaN NaN       NaN NaN
                            0.25    0.124326 NaN  0.073572 NaN
                            0.50    0.242571 NaN  0.159632 NaN
                            0.75    0.182210 NaN  0.109036 NaN
                            1.00    0.253949 NaN  0.169041 NaN
H-UNet-HPCA-T-FT   1.0      0.05    0.710282 NaN  0.582355 NaN
                            0.10    0.713232 NaN  0.594146 NaN
                            0.25    0.776007 NaN  0.660617 NaN
                            0.50    0.759104 NaN  0.649989 NaN
                            0.75    0.828377 NaN  0.719760 NaN
                            1.00    0.824910 NaN  0.721946 NaN
H-UNet-SWTA-FT     2.0      0.05    0.700188 NaN  0.563376 NaN
                            0.10    0.735819 NaN  0.613887 NaN
                            0.25    0.772984 NaN  0.652882 NaN
                            0.50    0.812226 NaN  0.701360 NaN
                            0.75    0.839314 NaN  0.737257 NaN
                            1.00    0.848543 NaN  0.749035 NaN
H-UNet-SWTA-T-FT   2.0      0.05    0.673676 NaN  0.538877 NaN
                            0.10    0.737468 NaN  0.604906 NaN
                            0.25    0.749083 NaN  0.627599 NaN
                            0.50    0.768846 NaN  0.657936 NaN
                            0.75    0.817256 NaN  0.708272 NaN
                            1.00    0.838178 NaN  0.735655 NaN
UNet               1.0      0.05    0.735901 NaN  0.605677 NaN
                            0.10    0.703565 NaN  0.568238 NaN
                            0.25    0.793149 NaN  0.677545 NaN
                            0.50    0.822572 NaN  0.717927 NaN
                            0.75    0.803457 NaN  0.694618 NaN
                            1.00    0.836816 NaN  0.735125 NaN

<h2>KvasirSEG Dataset</h2>

In [5]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_KvasirSEG=5          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=kvasirSEG/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_KvasirSEG)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_KvasirSEG)).glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=kvasirSEG/fcn32s/inv_temp-1').glob('regime-*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-swta_ft/inv_temp-{}'.format(INV_TEMP_KvasirSEG)).glob('regime-*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-swta_t_ft/inv_temp-{}'.format(INV_TEMP_KvasirSEG)).glob('regime-*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hfcn32s-hpca_t_ft/inv_temp-1').glob('regime-*')),
}

In [6]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [7]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,0.05,0.367204,0.244799
1,FCN32s,0,1.0,0.10,0.387325,0.259646
2,FCN32s,0,1.0,0.25,0.438789,0.305386
3,FCN32s,0,1.0,0.50,0.673402,0.552712
4,FCN32s,0,1.0,0.75,0.715107,0.597003
5,FCN32s,0,1.0,1.00,0.732239,0.615262
6,H-FCN32s-HPCA-FT,0,1.0,0.05,0.732153,0.620733
7,H-FCN32s-HPCA-FT,0,1.0,0.10,0.734247,0.623301
8,H-FCN32s-HPCA-FT,0,1.0,0.25,0.751306,0.644129
9,H-FCN32s-HPCA-FT,0,1.0,0.50,0.764707,0.659590


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      0.05    0.367204 NaN  0.244799 NaN
                            0.10    0.387325 NaN  0.259646 NaN
                            0.25    0.438789 NaN  0.305386 NaN
                            0.50    0.673402 NaN  0.552712 NaN
                            0.75    0.715107 NaN  0.597003 NaN
                            1.00    0.732239 NaN  0.615262 NaN
H-FCN32s-HPCA-FT   1.0      0.05    0.732153 NaN  0.620733 NaN
                            0.10    0.734247 NaN  0.623301 NaN
                            0.25    0.751306 NaN  0.644129 NaN
                            0.50    0.764707 NaN  0.659590 NaN
                            0.75    0.765654 NaN  0.656970 NaN
                            1.00    0.764679 NaN  0.658036 NaN
H-FCN32s-HPCA-T-FT 1.0      0.05    0.726625 NaN  0.615248 NaN
                            0.10    0.732137 NaN  0.621093 NaN
                            0.25    0.731264 NaN  0.624985 NaN
                            0.50    0.746005 NaN  0.636827 NaN
                            0.75    0.762028 NaN  0.656592 NaN
                            1.00    0.766556 NaN  0.660787 NaN
H-FCN32s-SWTA-FT   5.0      0.05    0.757021 NaN  0.648628 NaN
                            0.10    0.765693 NaN  0.657096 NaN
                            0.25    0.768939 NaN  0.663732 NaN
                            0.50    0.790153 NaN  0.685747 NaN
                            0.75    0.791474 NaN  0.685932 NaN
                            1.00    0.794742 NaN  0.690319 NaN
H-FCN32s-SWTA-T-FT 5.0      0.05    0.730225 NaN  0.614209 NaN
                            0.10    0.744027 NaN  0.632266 NaN
                            0.25    0.760167 NaN  0.649382 NaN
                            0.50    0.763040 NaN  0.652509 NaN
                            0.75    0.769005 NaN  0.661645 NaN
                            1.00    0.782000 NaN  0.676947 NaN
H-UNet-HPCA-FT     1.0      0.05    0.310613 NaN  0.200300 NaN
                            0.10    0.247744 NaN  0.153488 NaN
                            0.25    0.271499 NaN  0.170486 NaN
                            0.50    0.281318 NaN  0.175887 NaN
                            0.75    0.266302 NaN  0.166334 NaN
                            1.00    0.274454 NaN  0.173328 NaN
H-UNet-HPCA-T-FT   1.0      0.05    0.389038 NaN  0.263537 NaN
                            0.10    0.430399 NaN  0.295523 NaN
                            0.25    0.509006 NaN  0.371365 NaN
                            0.50    0.700316 NaN  0.572236 NaN
                            0.75    0.688045 NaN  0.558559 NaN
                            1.00    0.699386 NaN  0.574928 NaN
H-UNet-SWTA-FT     5.0      0.05    0.368309 NaN  0.251359 NaN
                            0.10    0.427420 NaN  0.299318 NaN
                            0.25    0.491097 NaN  0.355762 NaN
                            0.50    0.595646 NaN  0.458075 NaN
                            0.75    0.715248 NaN  0.590108 NaN
                            1.00    0.741005 NaN  0.620023 NaN
H-UNet-SWTA-T-FT   5.0      0.05    0.358164 NaN  0.234510 NaN
                            0.10    0.431327 NaN  0.298655 NaN
                            0.25    0.448814 NaN  0.312626 NaN
                            0.50    0.631098 NaN  0.495447 NaN
                            0.75    0.670861 NaN  0.538532 NaN
                            1.00    0.741180 NaN  0.624408 NaN
UNet               1.0      0.05    0.406711 NaN  0.275633 NaN
                            0.10    0.385883 NaN  0.260077 NaN
                            0.25    0.483924 NaN  0.345684 NaN
                            0.50    0.626224 NaN  0.493665 NaN
                            0.75    0.674666 NaN  0.546501 NaN
                            1.00    0.685022 NaN  0.560719 NaN

<h2>Data Science Bowl 2018 Dataset</h2>

In [16]:
#EXP_ROOT = "./runs"
EXP_ROOT = "/mnt/Results/hebbian-medical-image-segmentation/runs"

INV_TEMP_DataScienceBowl2018=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_DataScienceBowl2018)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_DataScienceBowl2018)).glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    # FCN32s-based
    'FCN32s': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/fcn32s/inv_temp-1').glob('regime-*')),
    'H-FCN32s-SWTA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-swta_ft/inv_temp-{}'.format(INV_TEMP_DataScienceBowl2018)).glob('regime-*')),
    'H-FCN32s-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-swta_t_ft/inv_temp-{}'.format(INV_TEMP_DataScienceBowl2018)).glob('regime-*')),
    'H-FCN32s-HPCA-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-FCN32s-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=datasciencebowl2018/hfcn32s-hpca_t_ft/inv_temp-1').glob('regime-*')),
}

In [17]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [18]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,FCN32s,0,1.0,0.05,0.199762,0.118341
1,FCN32s,0,1.0,0.10,0.453639,0.308424
2,FCN32s,0,1.0,0.25,0.580922,0.428747
3,FCN32s,0,1.0,0.50,0.702653,0.560443
4,FCN32s,0,1.0,0.75,0.707480,0.566368
5,FCN32s,0,1.0,1.00,0.746234,0.611988
6,H-FCN32s-HPCA-FT,0,1.0,0.05,0.716025,0.575547
7,H-FCN32s-HPCA-FT,0,1.0,0.10,0.725342,0.588237
8,H-FCN32s-HPCA-FT,0,1.0,0.25,0.736250,0.600037
9,H-FCN32s-HPCA-FT,0,1.0,0.50,0.761531,0.630555


Dice       Jaccard    
                                        mean std      mean std
Model              Inv Temp Regime                            
FCN32s             1.0      0.05    0.199762 NaN  0.118341 NaN
                            0.10    0.453639 NaN  0.308424 NaN
                            0.25    0.580922 NaN  0.428747 NaN
                            0.50    0.702653 NaN  0.560443 NaN
                            0.75    0.707480 NaN  0.566368 NaN
                            1.00    0.746234 NaN  0.611988 NaN
H-FCN32s-HPCA-FT   1.0      0.05    0.716025 NaN  0.575547 NaN
                            0.10    0.725342 NaN  0.588237 NaN
                            0.25    0.736250 NaN  0.600037 NaN
                            0.50    0.761531 NaN  0.630555 NaN
                            0.75    0.769257 NaN  0.640270 NaN
                            1.00    0.784804 NaN  0.660039 NaN
H-FCN32s-HPCA-T-FT 1.0      0.05    0.715732 NaN  0.574888 NaN
                            0.10    0.715543 NaN  0.575242 NaN
                            0.25    0.736035 NaN  0.600555 NaN
                            0.50    0.757217 NaN  0.626718 NaN
                            0.75    0.769595 NaN  0.641223 NaN
                            1.00    0.780666 NaN  0.654993 NaN
H-FCN32s-SWTA-FT   10.0     0.05    0.708557 NaN  0.566273 NaN
                            0.10    0.721243 NaN  0.581055 NaN
                            0.25    0.732337 NaN  0.595244 NaN
                            0.50    0.756250 NaN  0.624052 NaN
                            0.75    0.772159 NaN  0.643810 NaN
                            1.00    0.778396 NaN  0.651899 NaN
H-FCN32s-SWTA-T-FT 10.0     0.05    0.722416 NaN  0.582200 NaN
                            0.10    0.716955 NaN  0.577183 NaN
                            0.25    0.736557 NaN  0.600923 NaN
                            0.50    0.760665 NaN  0.629030 NaN
                            0.75    0.773439 NaN  0.645339 NaN
                            1.00    0.779170 NaN  0.652698 NaN
H-UNet-HPCA-FT     1.0      0.05    0.683152 NaN  0.534520 NaN
                            0.10    0.719348 NaN  0.573841 NaN
                            0.25    0.793767 NaN  0.673198 NaN
                            0.50    0.806803 NaN  0.688262 NaN
                            0.75    0.808288 NaN  0.691387 NaN
                            1.00    0.800751 NaN  0.679724 NaN
H-UNet-HPCA-T-FT   1.0      0.05    0.749689 NaN  0.613528 NaN
                            0.10    0.773193 NaN  0.646275 NaN
                            0.25    0.789517 NaN  0.669214 NaN
                            0.50    0.822787 NaN  0.708231 NaN
                            0.75    0.832363 NaN  0.721841 NaN
                            1.00    0.828157 NaN  0.716986 NaN
H-UNet-SWTA-FT     10.0     0.05    0.751929 NaN  0.619607 NaN
                            0.10    0.759428 NaN  0.631381 NaN
                            0.25    0.800772 NaN  0.684315 NaN
                            0.50    0.826116 NaN  0.718186 NaN
                            0.75    0.825479 NaN  0.715351 NaN
                            1.00    0.832328 NaN  0.729832 NaN
H-UNet-SWTA-T-FT   10.0     0.05    0.585926 NaN  0.451467 NaN
                            0.10    0.769839 NaN  0.643089 NaN
                            0.25    0.754752 NaN  0.622932 NaN
                            0.50    0.790548 NaN  0.674530 NaN
                            0.75    0.791473 NaN  0.683446 NaN
                            1.00    0.828268 NaN  0.717549 NaN
UNet               1.0      0.05    0.725875 NaN  0.591188 NaN
                            0.10    0.793178 NaN  0.671168 NaN
                            0.25    0.785602 NaN  0.663202 NaN
                            0.50    0.816926 NaN  0.704784 NaN
                            0.75    0.829068 NaN  0.718504 NaN
                            1.00    0.837379 NaN  0.729981 NaN